<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Hello-NARCIS" data-toc-modified-id="Hello-NARCIS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hello NARCIS</a></span></li><li><span><a href="#Preparation" data-toc-modified-id="Preparation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preparation</a></span><ul class="toc-item"><li><span><a href="#Config" data-toc-modified-id="Config-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Config</a></span></li><li><span><a href="#Start-Docker" data-toc-modified-id="Start-Docker-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Start Docker</a></span></li><li><span><a href="#Connect-to-MongoDB" data-toc-modified-id="Connect-to-MongoDB-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Connect to MongoDB</a></span></li></ul></li><li><span><a href="#Inventory-of-fields" data-toc-modified-id="Inventory-of-fields-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Inventory of fields</a></span><ul class="toc-item"><li><span><a href="#Top-level-fields" data-toc-modified-id="Top-level-fields-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Top level fields</a></span></li><li><span><a href="#Nested-fields" data-toc-modified-id="Nested-fields-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Nested fields</a></span></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Results</a></span></li><li><span><a href="#Appendix" data-toc-modified-id="Appendix-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Appendix</a></span><ul class="toc-item"><li><span><a href="#Use-variety" data-toc-modified-id="Use-variety-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Use <em>variety</em></a></span></li><li><span><a href="#Preparation" data-toc-modified-id="Preparation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Preparation</a></span></li><li><span><a href="#Run-variety" data-toc-modified-id="Run-variety-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Run <em>variety</em></a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Results</a></span></li></ul></li></ul></div>

## Hello NARCIS

We explore a dump of the [NARCIS](https://www.narcis.nl) data.
The dump has been taken bij 
[Emil Bode](https://dans.knaw.nl/nl/over/organisatie-beleid/medewerkers/bode)
in December 2017.
He has prepared it as MongoDB database contents in a Docker container.
This container is obtainable from DataVerse:
[NARCIS metadata in nldidlnorm format](http://hdl.handle.net/10411/FTZVH4).
Next to the dump is a *readme* that tells you exactly how to query the data locally.

The purpose of this notebook is an initial scan of the full database contents.
The database is called `NARCIS`, and in it is just one collection, `Dec2017`, with over 1,5 million
documents.

A document is a dictionary of keys and values, where values may be numbers, strings, dates, but also lists
of values and also dictionaries of keys and values.
The nesting of lists and dictionaries can be arbitrarily deep.

Documents do not have to conform to any schema. Most likely, there are a few dominant schemata,
but also likely: there might be outliers.

We assume no previous knowledge of NARCIS, so we want to explore the data from scratch.

This notebook performs a first step:
* We extract a distribution of all keys that are present in the docs of the database, together with
their frequencies.

## Preparation

We assume that you have a directory `local` under your home directory, with a directory `narcis` in it.
We assume that you have downloaded the dump from DataVerse and placed it under this `narcis` directory.

If you have it in a different location on your system, you can adapt the notebook to your own situation
by editing some values in the config section below.

You need to have [Docker](https://www.docker.com/get-docker) installed on your system.

In [ ]:
import collections
import os
import operator
from functools import reduce
from datetime import datetime, date
from pymongo import MongoClient
from bson.objectid import ObjectId

### Config

Specify the location of the database dump here; also the local location of this notebook.

In [60]:
DUMP_DIR = os.path.expanduser('~/local/narcis/MongoDB-NARCIS-Dec17')

REPO_DIR = os.path.expanduser('~/github/Dans-labs/narcis-explore')

TEMP_NAME = '_temp'
TEMP_DIR = f'{REPO_DIR}/{TEMP_NAME}'

RESULT_NAME = 'results'
RESULT_DIR = f'{REPO_DIR}/{RESULT_NAME}'

FIELDS_FILE_NAME = 'fields.tsv'
FIELDS_FILE = f'{RESULT_DIR}/{FIELDS_FILE_NAME}'
FIELDS_VAR_FILE_NAME = 'fields-variety.txt'
FIELDS_VAR_FILE = f'{RESULT_DIR}/{FIELDS_VAR_FILE_NAME}'

for outDir in (TEMP_DIR, RESULT_DIR):
    os.makedirs(outDir, exist_ok=True)

If we generate large files, we do so in a temporary directory `{{TEMP_NAME}}`, in the repository itself.
The repository lists `{{TEMP_NAME}}` in its `.gitignore` file, so output will not be sent to GitHub.

Result files will be generated in `{{RESULT_NAME}}`, also inside the repo. These files will be sent to GitHub.

### Start Docker

Below are the magic commands to start and stop the relevant Docker container.
They are shell commands, not Python commands.

In case you want to stop Docker

In [ ]:
!docker stop NARCIS
!docker rm NARCIS

In the next cell a Docker container is started.

In [ ]:
!docker run --name NARCIS -v {DUMP_DIR}:/data/db -p 27019:27017 -d mongo --logpath /data/db/log.log

### Connect to MongoDB
Now there is a MongoDb behind port 27019 that we can connect to.

We make the connection and get some very basic statistics about the contents of the database.

In [ ]:
client = MongoClient('mongodb://localhost:27019/')

In [ ]:
client.database_names()

Navigate to the Dec2017 collection

In [ ]:
DBN = client.NARCIS
DBN.collection_names()

In [ ]:
DBND = DBN.Dec2017
DBND.count()

## Inventory of fields

The next step is to have a look at the top-level fields of all the documents.

### Top level fields
We count how often each top-level field occurs.
We make separate counts for empty and non-empty values.

The troughput of docs is not very fast. The cell below takes  minutes. 

In [ ]:
nonEmptyFields = collections.Counter()
emptyFields = collections.Counter()

i = 0
j = 0
chunk = 100000

for doc in DBND.find():
    j += 1
    i += 1
    if j == chunk:
        j = 0
        print(f'\t{i:>7}')
    for (key, val) in doc.items():
        if val:
            nonEmptyFields[key] += 1
        else:
            emptyFields[key] += 1
print(f'\t{i:>7}')
print(f'There are {len(nonEmptyFields)} fields with a value')
print(f'There are {len(emptyFields)} fields without a value')

Here are the non empty fields and how often they occur:

In [ ]:
for (field, amount) in sorted(nonEmptyFields.items(), key=lambda x: (-x[1], x[0])):
    print(f'{field:<20} {amount:>7}x')

Every document in NARCIS has exactly the same set of top level fields.

### Nested fields

We want to explore a distribution of all fields, also the ones that occur
(deeply) nested in documents.

First we do it the Pythonic way: we request all documents from MongoDb, walk over them,
and extract their (nested) keys in a `collections.Counter()`.

It turns out that this takes 21 minutes.

Later we see that it is not straightforward to find a faster way, so we leave it at this.

In [ ]:
def getLeaves(key, val):
    if type(val) is list:
        return reduce(operator.add, (getLeaves(key, v) for v in val), ())
    elif type(val) is dict:
        return reduce(operator.add, (getLeaves(key + (ky,), v) for (ky, v) in val.items()), ())
    else:
        return (key,)

In [44]:
nestedFields = collections.Counter()
i = 0
j = 0
chunk = 100000
limit = -1

for doc in DBND.find():
    j += 1
    i += 1
    for k in getLeaves((), doc):
        nestedFields[k] += 1
    if j == chunk:
        j = 0
        print(f'\t{i:>7}')
    if limit > 0 and i >= limit:
        break
print(f'\t{i:>7}')
print(f'{len(nestedFields)} distinct keys encountered')

	 900000
	1000000
	1100000
	1200000
	1300000
	1400000
	1500000
	1600000
	1622397
1331 distinct keys encountered


We list the keys in order of frequency, most frequent ones first.
The result is writen to `{{FIELDS_FILE_NAME}}`.

In [61]:
with open(FIELDS_FILE, 'w') as fh:
    for (k, v) in sorted(nestedFields.items(), key=lambda x: (-x[1], x[0])):
        fh.write(f'{v:>7}\t{".".join(k)}\n')

In [62]:
!head -20 {FIELDS_FILE}

6464561	setSpec
3269404	nldidlnorm.Item.Component.Resource.mods.name._attrs
3244794	nldidlnorm.Component.Resource
3244794	nldidlnorm.Item.Component.Resource.mods._attrs
3244794	nldidlnorm.Item1.Component.Resource
3206882	nldidlnorm.Item.Component.Resource.mods.name.role.roleTerm._attrs
2883096	nldidlnorm.Item.Component.Resource.mods.name1._attrs
2745507	nldidlnorm.Item.Component.Resource.mods.name1.role.roleTerm._attrs
2630900	GlobalIDs
2546508	nldidlnorm.Item.Component.Resource.mods.language.languageTerm._attrs
2201504	nldidlnorm.Item.Component.Resource.mods.name2._attrs
2153322	nldidlnorm.Item.Component.Resource.mods.name2.role.roleTerm._attrs
1797998	Journal.relatedItem._attrs
1797998	nldidlnorm.Item.Component.Resource.mods.relatedItem._attrs
1659760	nldidlnorm.Item.Component.Resource.mods.originInfo.dateIssued._attrs
1629591	DAI
1622397	ID
1622397	NumberofIDs
1622397	_id
1622397	access


In [63]:
!tail -20 {FIELDS_FILE}

      1	nldidlnorm.Item.Component.Resource.mods.relatedItem1.part.detail._attrs
      1	nldidlnorm.Item.Component.Resource.mods.relatedItem1.part.detail.number
      1	nldidlnorm.Item.Component.Resource.mods.relatedItem1.part.detail1._attrs
      1	nldidlnorm.Item.Component.Resource.mods.relatedItem1.part.detail1.number
      1	nldidlnorm.Item.Component.Resource.mods.relatedItem4._attrs
      1	nldidlnorm.Item.Component.Resource.mods.relatedItem4.titleInfo.title
      1	nldidlnorm.Item.Component.Resource.mods.relatedItem5._attrs
      1	nldidlnorm.Item.Component.Resource.mods.relatedItem5.titleInfo.title
      1	nldidlnorm.Item.Component.Resource.mods.subject0.topic0
      1	nldidlnorm.Item.Component.Resource.mods.subject0.topic8
      1	nldidlnorm.Item.Component.Resource.mods.subject0.topic9
      1	nldidlnorm.Item0.Descriptor1.Statement.modified
      1	nldidlnorm.Item0.Descriptor3.Statement.description
      1	nldidlnorm.Item5.Descriptor1.Statement.modified
      1	nldidlnorm.Item7.

## Results
The product of this notebook is the file `{{FIELDS_FILE_NAME}}` that shows us all the possible, nested keys in the
NARCIS database with their frequencies.
In other notebooks we will use this file, and conduct further explorations.

## Appendix

The rest of this notebook is an alternative approach that we will not build on.

### Use *variety*

Normally, you can speed up data processing by having the database use its query engine to its full potential.
Using `mapReduce` inside MongoDb as indicated on
[stack overflow](https://stackoverflow.com/questions/2298870/mongodb-get-names-of-all-keys-in-collection)
could do the trick. But the accepted answer there does not do  nested keys.
So we have to write a MongoDb function for mapReduce, which means writing something in Javascript.

It turns out that somebody has already done this, and made a nice library out of it:
[variety](https://github.com/variety/variety). 
This is a tool to explore the dominant scheme of a MongoDB and its outlier documents.

However, the performance is much worse than the previous method:
going this way takes you a whopping 80 minutes.
We only show how to do it, generate the results, but we do not recommend it.

If you want to try it yourself, there is extra preparation to do.

### Preparation

You must have *MongoDb* and *Node* installed.

Install *variety*:

```
npm install variety-cli -g
```

### Run *variety*
This is a javascript program; we need Node to run it.
The next cell takes well over an hour!

In [ ]:
!variety --port=27019 NARCIS/Dec2017

### Results

We have copied the output into the file `{{FIELDS_VAR_FILE_NAME}}`, so it does not get inadvertently lost.
After saving the output, we discarded it from the cell.

The results are pleasing to the eye (if your screen is wide enough) but
they lend themselves less well for further processing.

In [64]:
!head -20 {FIELDS_VAR_FILE}

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| key                                                                                       | types                           | occurrences | percents                 |
| ----------------------------------------------------------------------------------------- | ------------------------------- | ----------- | ------------------------ |
| DAI                                                                                       | Array                           |     1622397 | 100.00000000000000000000 |
| GlobalIDs                                                                                 | Array                           |     1622397 | 100.00000000000000000000 |
| ID                                                                                        | Array                           |     1622397 | 100.000000000

In [65]:
!tail -40 {FIELDS_VAR_FILE}

| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem4.titleInfo.title                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5                                   | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5._attrs                            | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5.titleInfo                         | Object                          |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.relatedItem5.titleInfo.title                   | Array                           |           1 |   0.00006163719484195299 |
| nldidlnorm.XX.Item.Component.Resource.mods.subject0.topic0                                | Array                           |           1 |   0.000061637